In [3]:
import onnx
import onnx.helper as helper
import numpy as np
from onnx import numpy_helper

def create_simple_animal_classifier():
    """创建一个简单的动物分类ONNX模型"""
    
    # 模型输入：1x3x224x224 (batch, channels, height, width)
    input_tensor = helper.make_tensor_value_info('input', onnx.TensorProto.FLOAT, [1, 3, 224, 224])
    
    # 模型输出：1x10 (10个动物类别)
    output_tensor = helper.make_tensor_value_info('output', onnx.TensorProto.FLOAT, [1, 10])
    
    # 创建一些简单的操作节点（模拟一个分类器）
    nodes = [
        # 全局平均池化
        helper.make_node(
            'GlobalAveragePool',
            inputs=['input'],
            outputs=['pool_output'],
            name='global_pool'
        ),
        
        # 全连接层（模拟分类器）
        helper.make_node(
            'Gemm',
            inputs=['pool_output', 'fc_weight', 'fc_bias'],
            outputs=['output'],
            name='fc_layer'
        )
    ]
    
    # 创建权重和偏置（随机初始化）
    fc_weight = numpy_helper.from_array(
        np.random.randn(10, 3).astype(np.float32), 
        name='fc_weight'
    )
    
    fc_bias = numpy_helper.from_array(
        np.random.randn(10).astype(np.float32), 
        name='fc_bias'
    )
    
    # 创建图
    graph = helper.make_graph(
        nodes=nodes,
        name='animal_classifier',
        inputs=[input_tensor],
        outputs=[output_tensor],
        initializer=[fc_weight, fc_bias]
    )
    
    # 创建模型
    model = helper.make_model(
        graph,
        producer_name='animal-classifier',
        opset_imports=[helper.make_opsetid("", 12)]
    )
    
    return model

# 生成并保存模型
model = create_simple_animal_classifier()
onnx.save(model, 'animal_classifier.onnx')
print("ONNX模型已生成: animal_classifier.onnx")

ONNX模型已生成: animal_classifier.onnx


In [4]:
# 生成动物类别标签文件
animal_labels = [
    "非洲象", "亚洲象", "狮子", "老虎", "长颈鹿",
    "斑马", "大熊猫", "北极熊", "猎豹", "犀牛"
]

with open('animal_labels.txt', 'w', encoding='utf-8') as f:
    for label in animal_labels:
        f.write(label + '\n')

print("类别标签文件已生成: animal_labels.txt")
print("包含的动物类别:")
for i, animal in enumerate(animal_labels):
    print(f"{i}: {animal}")

类别标签文件已生成: animal_labels.txt
包含的动物类别:
0: 非洲象
1: 亚洲象
2: 狮子
3: 老虎
4: 长颈鹿
5: 斑马
6: 大熊猫
7: 北极熊
8: 猎豹
9: 犀牛


In [5]:
from PIL import Image, ImageDraw
import random

def create_test_animal_image():
    """创建一个模拟的动物测试图片"""
    # 创建一个224x224的图片
    img = Image.new('RGB', (224, 224), color='lightblue')
    draw = ImageDraw.Draw(img)
    
    # 随机选择一个动物类型来绘制简单的形状
    animal_type = random.choice(['elephant', 'lion', 'giraffe', 'zebra'])
    
    if animal_type == 'elephant':
        # 绘制大象的简单轮廓
        draw.ellipse([50, 80, 170, 180], fill='gray', outline='black')  # 身体
        draw.ellipse([30, 100, 70, 140], fill='gray', outline='black')  # 头部
        draw.ellipse([20, 110, 40, 120], fill='gray', outline='black')  # 耳朵
        draw.ellipse([60, 110, 80, 120], fill='gray', outline='black')  # 耳朵
        draw.line([40, 140, 20, 160], fill='black', width=3)  # 鼻子
        
    elif animal_type == 'lion':
        # 绘制狮子的简单轮廓
        draw.ellipse([80, 80, 160, 160], fill='goldenrod', outline='black')  # 身体
        draw.ellipse([70, 70, 170, 120], fill='darkgoldenrod', outline='black')  # 鬃毛
        draw.ellipse([90, 90, 110, 110], fill='black')  # 眼睛
        draw.ellipse([130, 90, 150, 110], fill='black')  # 眼睛
        
    elif animal_type == 'giraffe':
        # 绘制长颈鹿的简单轮廓
        draw.rectangle([100, 120, 120, 180], fill='yellow', outline='black')  # 身体
        draw.rectangle([95, 80, 105, 120], fill='yellow', outline='black')  # 脖子
        draw.ellipse([90, 70, 110, 90], fill='yellow', outline='black')  # 头部
        draw.ellipse([95, 75, 100, 80], fill='black')  # 眼睛
        
    elif animal_type == 'zebra':
        # 绘制斑马的简单轮廓
        draw.ellipse([80, 100, 160, 180], fill='white', outline='black')  # 身体
        # 添加条纹
        for i in range(85, 155, 10):
            draw.rectangle([i, 100, i+5, 180], fill='black')
    
    # 添加一些背景元素
    draw.rectangle([10, 190, 214, 200], fill='green')  # 草地
    
    return img, animal_type

# 生成测试图片
test_image, animal_type = create_test_animal_image()
test_image.save('test_animal.jpg')
print(f"测试图片已生成: test_animal.jpg (模拟动物: {animal_type})")

测试图片已生成: test_animal.jpg (模拟动物: lion)


In [8]:
def verify_generated_files():
    """验证生成的文件是否可用"""
    print("\n=== 验证生成的文件 ===")
    
    try:
        # 验证ONNX模型
        import onnxruntime as ort
        session = ort.InferenceSession('animal_classifier.onnx')
        print("✓ ONNX模型加载成功")
        print(f"  输入: {session.get_inputs()[0].name}")
        print(f"  输出: {session.get_outputs()[0].name}")
        
        # 验证标签文件
        with open('animal_labels.txt', 'r', encoding='utf-8') as f:
            labels = [line.strip() for line in f.readlines()]
        print(f"✓ 标签文件加载成功，共{len(labels)}个类别")
        
        # 验证图片文件
        from PIL import Image
        image = Image.open('test_animal.jpg')
        print(f"✓ 测试图片加载成功，尺寸: {image.size}")
        
        print("\n所有文件验证通过！可以开始做题了。")
        
    except Exception as e:
        print(f"✗ 文件验证失败: {e}")

# 运行验证
verify_generated_files()


=== 验证生成的文件 ===
✗ 文件验证失败: [ONNXRuntimeError] : 1 : FAIL : Load model from animal_classifier.onnx failed:Node (fc_layer) Op (Gemm) [ShapeInferenceError] First input does not have rank 2


In [9]:
import onnx
import onnx.helper as helper
import numpy as np
from onnx import numpy_helper
from PIL import Image, ImageDraw

def generate_onnx_model_files():
    """生成ONNX模型推理题目所需的所有文件"""
    
    print("开始生成ONNX模型推理题目文件...")
    
    # 1. 生成ONNX模型 - 修复版本
    print("1. 生成ONNX模型...")
    def create_simple_animal_classifier():
        # 模型输入：1x3x224x224 (batch, channels, height, width)
        input_tensor = helper.make_tensor_value_info('input', onnx.TensorProto.FLOAT, [1, 3, 224, 224])
        
        # 模型输出：1x10 (10个动物类别)
        output_tensor = helper.make_tensor_value_info('output', onnx.TensorProto.FLOAT, [1, 10])
        
        # 创建更简单的模型结构
        nodes = [
            # 全局平均池化 - 输出形状: [1, 3, 1, 1]
            helper.make_node(
                'GlobalAveragePool',
                inputs=['input'],
                outputs=['pool_output'],
                name='global_pool'
            ),
            
            # 重塑为 [1, 3] 以匹配全连接层
            helper.make_node(
                'Reshape',
                inputs=['pool_output', 'reshape_shape'],
                outputs=['reshape_output'],
                name='reshape'
            ),
            
            # 全连接层
            helper.make_node(
                'Gemm',
                inputs=['reshape_output', 'fc_weight', 'fc_bias'],
                outputs=['output'],
                name='fc_layer'
            )
        ]
        
        # 创建常量张量
        reshape_shape = numpy_helper.from_array(
            np.array([1, 3], dtype=np.int64), 
            name='reshape_shape'
        )
        
        # 创建权重和偏置（随机初始化）
        fc_weight = numpy_helper.from_array(
            np.random.randn(10, 3).astype(np.float32), 
            name='fc_weight'
        )
        
        fc_bias = numpy_helper.from_array(
            np.random.randn(10).astype(np.float32), 
            name='fc_bias'
        )
        
        # 创建图
        graph = helper.make_graph(
            nodes=nodes,
            name='animal_classifier',
            inputs=[input_tensor],
            outputs=[output_tensor],
            initializer=[reshape_shape, fc_weight, fc_bias]
        )
        
        # 创建模型
        model = helper.make_model(
            graph,
            producer_name='animal-classifier',
            opset_imports=[helper.make_opsetid("", 13)]
        )
        
        # 检查模型
        onnx.checker.check_model(model)
        
        return model
    
    try:
        model = create_simple_animal_classifier()
        onnx.save(model, 'animal_classifier.onnx')
        print("✓ ONNX模型生成成功")
    except Exception as e:
        print(f"✗ ONNX模型生成失败: {e}")
        return
    
    # 2. 生成类别标签
    print("2. 生成类别标签文件...")
    animal_labels = [
        "非洲象", "亚洲象", "狮子", "老虎", "长颈鹿",
        "斑马", "大熊猫", "北极熊", "猎豹", "犀牛"
    ]
    
    with open('animal_labels.txt', 'w', encoding='utf-8') as f:
        for label in animal_labels:
            f.write(label + '\n')
    print("✓ 类别标签文件生成成功")
    
    # 3. 生成测试图片
    print("3. 生成测试图片...")
    def create_test_image():
        # 创建一个224x224的彩色图片
        img = Image.new('RGB', (224, 224), color=(173, 216, 230))  # 浅蓝色背景
        draw = ImageDraw.Draw(img)
        
        # 绘制一个简单的动物轮廓（大象）
        # 身体
        draw.ellipse([60, 100, 164, 180], fill=(128, 128, 128), outline=(0, 0, 0), width=2)
        # 头部
        draw.ellipse([40, 80, 100, 120], fill=(128, 128, 128), outline=(0, 0, 0), width=2)
        # 耳朵
        draw.ellipse([30, 85, 50, 105], fill=(128, 128, 128), outline=(0, 0, 0), width=1)
        draw.ellipse([90, 85, 110, 105], fill=(128, 128, 128), outline=(0, 0, 0), width=1)
        # 鼻子
        draw.rectangle([45, 110, 55, 130], fill=(128, 128, 128), outline=(0, 0, 0), width=1)
        # 腿
        draw.rectangle([70, 170, 80, 190], fill=(128, 128, 128), outline=(0, 0, 0), width=1)
        draw.rectangle([90, 170, 100, 190], fill=(128, 128, 128), outline=(0, 0, 0), width=1)
        draw.rectangle([124, 170, 134, 190], fill=(128, 128, 128), outline=(0, 0, 0), width=1)
        draw.rectangle([144, 170, 154, 190], fill=(128, 128, 128), outline=(0, 0, 0), width=1)
        # 草地
        draw.rectangle([0, 190, 224, 200], fill=(34, 139, 34), outline=(0, 0, 0), width=1)
        
        return img
    
    try:
        test_image = create_test_image()
        test_image.save('test_animal.jpg', quality=95)
        print("✓ 测试图片生成成功")
    except Exception as e:
        print(f"✗ 测试图片生成失败: {e}")
        return
    
    print("\n=== 文件生成完成 ===")
    print("生成的文件:")
    print("1. animal_classifier.onnx - ONNX模型文件")
    print("2. animal_labels.txt - 动物类别标签")
    print("3. test_animal.jpg - 测试图片")
    
    print("\n动物类别列表:")
    for i, animal in enumerate(animal_labels):
        print(f"   {i}: {animal}")

# 执行生成
if __name__ == "__main__":
    generate_onnx_model_files()

开始生成ONNX模型推理题目文件...
1. 生成ONNX模型...
✓ ONNX模型生成成功
2. 生成类别标签文件...
✓ 类别标签文件生成成功
3. 生成测试图片...
✓ 测试图片生成成功

=== 文件生成完成 ===
生成的文件:
1. animal_classifier.onnx - ONNX模型文件
2. animal_labels.txt - 动物类别标签
3. test_animal.jpg - 测试图片

动物类别列表:
   0: 非洲象
   1: 亚洲象
   2: 狮子
   3: 老虎
   4: 长颈鹿
   5: 斑马
   6: 大熊猫
   7: 北极熊
   8: 猎豹
   9: 犀牛


In [11]:
def test_complete_inference():
    """测试完整的推理流程"""
    print("\n=== 测试完整推理流程 ===")
    
    try:
        import onnxruntime as ort
        import numpy as np
        from PIL import Image
        import scipy.special
        
        # 1. 加载模型
        session = ort.InferenceSession('animal_classifier.onnx')
        input_name = session.get_inputs()[0].name
        output_name = session.get_outputs()[0].name
        
        # 2. 加载标签
        with open('animal_labels.txt', 'r', encoding='utf-8') as f:
            labels = [line.strip() for line in f.readlines()]
        
        # 3. 预处理函数
        def preprocess_image(image_path, target_size=(224, 224)):
            image = Image.open(image_path).convert('RGB')
            image = image.resize(target_size)
            image_array = np.array(image).astype(np.float32)
            image_array = image_array / 255.0
            mean = [0.485, 0.456, 0.406]
            std = [0.229, 0.224, 0.225]
            image_array = (image_array - mean) / std
            image_array = np.transpose(image_array, (2, 0, 1))
            image_array = np.expand_dims(image_array, axis=0)
            return image_array
        
        # 4. 预处理图像
        processed_image = preprocess_image('test_animal.jpg')
        
        # 5. 执行推理
        outputs = session.run([output_name], {input_name: processed_image})
        
        # 6. 后处理
        probabilities = scipy.special.softmax(outputs[0], axis=1)
        predicted_class_idx = np.argmax(probabilities[0])
        confidence = probabilities[0][predicted_class_idx]
        predicted_label = labels[predicted_class_idx]
        
        print(f"预测结果: {predicted_label}")
        print(f"置信度: {confidence:.4f}")
        
        # 7. Top-3 结果
        top3_indices = np.argsort(probabilities[0])[-3:][::-1]
        top3_confidences = probabilities[0][top3_indices]
        
        print("\nTop-3 预测结果:")
        for i, (idx, conf) in enumerate(zip(top3_indices, top3_confidences)):
            print(f"{i+1}. {labels[idx]} - {conf:.4f}")
            
        print("\n🎉 完整推理流程测试成功！")
        
    except Exception as e:
        print(f"✗ 推理流程测试失败: {e}")

# 运行完整测试
test_complete_inference()


=== 测试完整推理流程 ===
✗ 推理流程测试失败: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Unexpected input data type. Actual: (tensor(double)) , expected: (tensor(float))


In [12]:
import onnx
import onnx.helper as helper
import numpy as np
from onnx import numpy_helper
from PIL import Image, ImageDraw

def generate_onnx_model_files():
    """生成ONNX模型推理题目所需的所有文件"""
    
    print("开始生成ONNX模型推理题目文件...")
    
    # 1. 生成ONNX模型 - 修复数据类型问题
    print("1. 生成ONNX模型...")
    def create_simple_animal_classifier():
        # 模型输入：1x3x224x224 (batch, channels, height, width)
        input_tensor = helper.make_tensor_value_info('input', onnx.TensorProto.FLOAT, [1, 3, 224, 224])
        
        # 模型输出：1x10 (10个动物类别)
        output_tensor = helper.make_tensor_value_info('output', onnx.TensorProto.FLOAT, [1, 10])
        
        # 创建更简单的模型结构 - 避免复杂的操作
        nodes = [
            # 使用卷积层替代全连接层，避免形状问题
            helper.make_node(
                'Conv',
                inputs=['input', 'conv_weight', 'conv_bias'],
                outputs=['conv_output'],
                name='conv_layer',
                kernel_shape=[1, 1],  # 1x1卷积，相当于全连接
                strides=[1, 1],
                pads=[0, 0, 0, 0]
            ),
            
            # 全局平均池化
            helper.make_node(
                'GlobalAveragePool',
                inputs=['conv_output'],
                outputs=['pool_output'],
                name='global_pool'
            ),
            
            # 重塑为正确的输出形状
            helper.make_node(
                'Reshape',
                inputs=['pool_output', 'reshape_shape'],
                outputs=['output'],
                name='reshape_output'
            )
        ]
        
        # 创建权重 - 确保是float32类型
        conv_weight = numpy_helper.from_array(
            np.random.randn(10, 3, 1, 1).astype(np.float32),  # 10个输出通道，3个输入通道，1x1卷积核
            name='conv_weight'
        )
        
        conv_bias = numpy_helper.from_array(
            np.random.randn(10).astype(np.float32),
            name='conv_bias'
        )
        
        # 重塑形状
        reshape_shape = numpy_helper.from_array(
            np.array([1, 10], dtype=np.int64),
            name='reshape_shape'
        )
        
        # 创建图
        graph = helper.make_graph(
            nodes=nodes,
            name='animal_classifier',
            inputs=[input_tensor],
            outputs=[output_tensor],
            initializer=[conv_weight, conv_bias, reshape_shape]
        )
        
        # 创建模型
        model = helper.make_model(
            graph,
            producer_name='animal-classifier',
            opset_imports=[helper.make_opsetid("", 13)]
        )
        
        # 检查模型
        onnx.checker.check_model(model)
        
        return model
    
    try:
        model = create_simple_animal_classifier()
        onnx.save(model, 'animal_classifier.onnx')
        print("✓ ONNX模型生成成功")
    except Exception as e:
        print(f"✗ ONNX模型生成失败: {e}")
        return
    
    # 2. 生成类别标签
    print("2. 生成类别标签文件...")
    animal_labels = [
        "非洲象", "亚洲象", "狮子", "老虎", "长颈鹿",
        "斑马", "大熊猫", "北极熊", "猎豹", "犀牛"
    ]
    
    with open('animal_labels.txt', 'w', encoding='utf-8') as f:
        for label in animal_labels:
            f.write(label + '\n')
    print("✓ 类别标签文件生成成功")
    
    # 3. 生成测试图片
    print("3. 生成测试图片...")
    def create_test_image():
        # 创建一个224x224的彩色图片
        img = Image.new('RGB', (224, 224), color=(173, 216, 230))  # 浅蓝色背景
        draw = ImageDraw.Draw(img)
        
        # 绘制一个简单的动物轮廓（大象）
        # 身体
        draw.ellipse([60, 100, 164, 180], fill=(128, 128, 128), outline=(0, 0, 0), width=2)
        # 头部
        draw.ellipse([40, 80, 100, 120], fill=(128, 128, 128), outline=(0, 0, 0), width=2)
        # 耳朵
        draw.ellipse([30, 85, 50, 105], fill=(128, 128, 128), outline=(0, 0, 0), width=1)
        draw.ellipse([90, 85, 110, 105], fill=(128, 128, 128), outline=(0, 0, 0), width=1)
        # 鼻子
        draw.rectangle([45, 110, 55, 130], fill=(128, 128, 128), outline=(0, 0, 0), width=1)
        # 腿
        draw.rectangle([70, 170, 80, 190], fill=(128, 128, 128), outline=(0, 0, 0), width=1)
        draw.rectangle([90, 170, 100, 190], fill=(128, 128, 128), outline=(0, 0, 0), width=1)
        draw.rectangle([124, 170, 134, 190], fill=(128, 128, 128), outline=(0, 0, 0), width=1)
        draw.rectangle([144, 170, 154, 190], fill=(128, 128, 128), outline=(0, 0, 0), width=1)
        # 草地
        draw.rectangle([0, 190, 224, 200], fill=(34, 139, 34), outline=(0, 0, 0), width=1)
        
        return img
    
    try:
        test_image = create_test_image()
        test_image.save('test_animal.jpg', quality=95)
        print("✓ 测试图片生成成功")
    except Exception as e:
        print(f"✗ 测试图片生成失败: {e}")
        return
    
    print("\n=== 文件生成完成 ===")
    print("生成的文件:")
    print("1. animal_classifier.onnx - ONNX模型文件")
    print("2. animal_labels.txt - 动物类别标签")
    print("3. test_animal.jpg - 测试图片")
    
    print("\n动物类别列表:")
    for i, animal in enumerate(animal_labels):
        print(f"   {i}: {animal}")

# 执行生成
if __name__ == "__main__":
    generate_onnx_model_files()

开始生成ONNX模型推理题目文件...
1. 生成ONNX模型...
✓ ONNX模型生成成功
2. 生成类别标签文件...
✓ 类别标签文件生成成功
3. 生成测试图片...
✓ 测试图片生成成功

=== 文件生成完成 ===
生成的文件:
1. animal_classifier.onnx - ONNX模型文件
2. animal_labels.txt - 动物类别标签
3. test_animal.jpg - 测试图片

动物类别列表:
   0: 非洲象
   1: 亚洲象
   2: 狮子
   3: 老虎
   4: 长颈鹿
   5: 斑马
   6: 大熊猫
   7: 北极熊
   8: 猎豹
   9: 犀牛


In [13]:
def test_complete_inference():
    """测试完整的推理流程 - 修复数据类型问题"""
    print("\n=== 测试完整推理流程 ===")
    
    try:
        import onnxruntime as ort
        import numpy as np
        from PIL import Image
        import scipy.special
        
        # 1. 加载模型
        session = ort.InferenceSession('animal_classifier.onnx')
        input_name = session.get_inputs()[0].name
        output_name = session.get_outputs()[0].name
        
        # 2. 加载标签
        with open('animal_labels.txt', 'r', encoding='utf-8') as f:
            labels = [line.strip() for line in f.readlines()]
        
        # 3. 预处理函数 - 确保输出是float32
        def preprocess_image(image_path, target_size=(224, 224)):
            image = Image.open(image_path).convert('RGB')
            image = image.resize(target_size)
            image_array = np.array(image).astype(np.float32)  # 确保是float32
            image_array = image_array / 255.0
            mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)  # 确保mean是float32
            std = np.array([0.229, 0.224, 0.225], dtype=np.float32)   # 确保std是float32
            image_array = (image_array - mean) / std
            image_array = np.transpose(image_array, (2, 0, 1))
            image_array = np.expand_dims(image_array, axis=0)
            return image_array
        
        # 4. 预处理图像
        processed_image = preprocess_image('test_animal.jpg')
        print(f"预处理图像数据类型: {processed_image.dtype}")
        print(f"预处理图像形状: {processed_image.shape}")
        
        # 5. 执行推理
        outputs = session.run([output_name], {input_name: processed_image})
        
        # 6. 后处理
        probabilities = scipy.special.softmax(outputs[0], axis=1)
        predicted_class_idx = np.argmax(probabilities[0])
        confidence = probabilities[0][predicted_class_idx]
        predicted_label = labels[predicted_class_idx]
        
        print(f"预测结果: {predicted_label}")
        print(f"置信度: {confidence:.4f}")
        
        # 7. Top-3 结果
        top3_indices = np.argsort(probabilities[0])[-3:][::-1]
        top3_confidences = probabilities[0][top3_indices]
        
        print("\nTop-3 预测结果:")
        for i, (idx, conf) in enumerate(zip(top3_indices, top3_confidences)):
            print(f"{i+1}. {labels[idx]} - {conf:.4f}")
            
        print("\n🎉 完整推理流程测试成功！")
        
    except Exception as e:
        print(f"✗ 推理流程测试失败: {e}")
        import traceback
        traceback.print_exc()

# 运行完整测试
test_complete_inference()


=== 测试完整推理流程 ===
预处理图像数据类型: float32
预处理图像形状: (1, 3, 224, 224)
预测结果: 老虎
置信度: 0.4131

Top-3 预测结果:
1. 老虎 - 0.4131
2. 非洲象 - 0.2929
3. 长颈鹿 - 0.1967

🎉 完整推理流程测试成功！
